### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import re
import datetime
import copy

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Semi-Monthly Closed Invoice\2022 Semi-Monthly Closed Invoice\2022 Semi-Monthly Report\Semi-Monthly Report Pallet Qty'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

In [ ]:
acc = excel_list[0]['Sheet1'][['WISE Facility', 'Item', 'Billing Period Start',  'BillingCode', 'Qty']]
acc = acc.rename(columns={'Qty':'Billing Qty'})
acc

In [ ]:
acc_cus = acc.loc[acc['Item'] == 'Customized Size Pallet']
acc_cus['Billing Pallet Type'] = np.nan

for i in range(len(acc_cus['BillingCode'])):   
    p = acc_cus['BillingCode'].iloc[i].split(',')[-1].replace(';', '').replace(' ', '').partition('pallet')[-1]    
    pp = re.sub('[x][0-9][0-9]', lambda x: x[0] + ' ', p)    
    ppp = re.sub('^[t]', '', pp)    
    pppp = re.sub('\s$', '', ppp)    
    acc_cus['Billing Pallet Type'].iloc[i] = pppp

acc_cus

In [ ]:
np.unique(acc_cus['Billing Pallet Type'])

In [ ]:
acc_a = acc.loc[acc['Item'] == 'Grade A Pallet']
acc_a['Billing Pallet Type'] = '48x40 #1'

acc_b = acc.loc[acc['Item'] == 'Grade B Pallet']
acc_b['Billing Pallet Type'] = '48x40 #2'

acc_cb = acc_a.append(acc_b).append(acc_cus)
acc_cb = acc_cb.sort_values(['WISE Facility', 'Billing Period Start'])
acc_cb

In [ ]:
np.unique(acc_cb['WISE Facility'])

In [ ]:
df_acc = acc_cb.groupby(['WISE Facility', 'Item', 'Billing Period Start', 
                         'Billing Pallet Type'])['Billing Qty'].sum().reset_index()
df_acc

In [ ]:
pal = excel_list[1]['Sheet1'][['WISE Facility', 'Order Date', 'Pallet Type', 'Description', 'Order Qty']]
pal = pal.rename(columns={'Pallet Type':'Purchase Pallet Type', 'Order Date':'Purchase Order Date'})

# pal['Received Qty'] = pal['Received Qty'].replace(0, np.nan)

# pal = pal.dropna(subset=['Facility', 'Received Qty'])

pal['Purchase Order Date'] = pal['Purchase Order Date'].dt.month_name(locale='English').str.slice(stop=3)
pal['Purchase Pallet Type'] = pal['Purchase Pallet Type'].str.lower()
pal['WISE Facility'] = pal['WISE Facility'].replace(['Buena Park', 'GP Texas', 'Memphis 2', 'Portage', 'NJ'],
                                          ['Valley View', 'Grand Prairie', 'Quality-4400', 'Indiana', 'New Jersey'])

pal

In [ ]:
np.unique(pal['WISE Facility'])

In [ ]:
pal['Item'] = np.nan

for i in range(len(pal['Purchase Pallet Type'])):
    if pal['Purchase Pallet Type'].iloc[i] == '48x40 #1':
        pal['Item'].iloc[i] = 'Grade A Pallet'
        
    elif pal['Purchase Pallet Type'].iloc[i] == '48x40 #2':
        pal['Item'].iloc[i] = 'Grade B Pallet'  
        
    else:
        pal['Item'].iloc[i] = 'Customized Size Pallet'        
        
pal

In [ ]:
df_pal = pal.groupby(['WISE Facility', 'Item', 'Purchase Order Date', 'Purchase Pallet Type'])['Order Qty'].sum().reset_index()
df_pal

In [ ]:
df_acc['Billing Pallet Type'].isin(df_pal['Purchase Pallet Type']).value_counts()

In [ ]:
df_cb = pd.merge(df_acc, df_pal, left_on = ['WISE Facility', 'Billing Period Start', 'Billing Pallet Type', 'Item'],
                right_on = ['WISE Facility', 'Purchase Order Date', 'Purchase Pallet Type', 'Item'], how = 'outer')
df_cb

In [ ]:
# df_cb.to_excel("012021_112021 Billing and Purchase Pallet.xlsx", index = False, header = True)

### Remove block and stringer

In [ ]:
acc_cb2 = copy.deepcopy(acc_cb)
pal2 = copy.deepcopy(pal)

In [ ]:
np.unique(acc_cb2['Billing Pallet Type'])

In [ ]:
for i in range(len(acc_cb2['Billing Pallet Type'])):
    
    acc_cb2['Billing Pallet Type'].iloc[i] = re.sub('\s[a-z]+', '', acc_cb2['Billing Pallet Type'].iloc[i])
    acc_cb2['Billing Pallet Type'].iloc[i] = re.sub('[a-z]+90x40', '90x40', acc_cb2['Billing Pallet Type'].iloc[i])

np.unique(acc_cb2['Billing Pallet Type'])

In [ ]:
df_acc2 = acc_cb2.groupby(['WISE Facility', 'Item', 'Billing Period Start', 
                         'Billing Pallet Type'])['Billing Qty'].sum().reset_index()
df_acc2

In [ ]:
np.unique(pal2['Purchase Pallet Type'])

In [ ]:
for i in range(len(pal2['Purchase Pallet Type'])):
    
    pal2['Purchase Pallet Type'].iloc[i] = re.sub('\s[a-z]+', '', pal2['Purchase Pallet Type'].iloc[i])

np.unique(pal2['Purchase Pallet Type'])

In [ ]:
df_pal2 = pal2.groupby(['WISE Facility', 'Item', 'Purchase Order Date', 'Purchase Pallet Type'])['Order Qty'].sum().reset_index()
df_pal2

In [ ]:
df_cb2 = pd.merge(df_acc2, df_pal2, left_on = ['WISE Facility', 'Billing Period Start', 'Billing Pallet Type', 'Item'],
                right_on = ['WISE Facility', 'Purchase Order Date', 'Purchase Pallet Type', 'Item'], how = 'outer')
df_cb2

In [ ]:
df_cb2.to_excel("2022 Billing and Purchase Pallet.xlsx", index = False, header = True)